In [2]:
import pickle as pkl
import numpy as np
import os
import torch

In [9]:
ids = []
for i in range(10):
    print(f"big loop: {i}")
    folder = os.path.join('./data', 'submit', 'affine', str(1), f'partition_{i}')
    with open(f'{folder}/A_train', 'rb') as f:
        A_train_reps = pkl.load(f)
    with open(f'{folder}/B_train', 'rb') as f:
        B_train_reps = pkl.load(f)
    with open(f'{folder}/B_test', 'rb') as f:
        B_test_reps = pkl.load(f)
    with open(f'{folder}/ids_train', 'rb') as f:
        ids_train = pkl.load(f)
    with open(f'{folder}/ids_test', 'rb') as f:
        ids_test = pkl.load(f)

    ids_iter = np.concatenate([ids_train, ids_test], axis=0)
    ids.append(ids_iter)

dct = np.load("./task2_submission.npz")

ids = np.concatenate(ids, axis=0)
preds = dct['representations']

print(type(ids[0]))
print(type(dct['ids'][0]))

id_rep_map = {id: rep for id, rep in zip(ids, preds)}

representations = [id_rep_map[id] for id in dct["ids"]]

np.savez("./task2_submission2.npz", ids=dct["ids"], representations=representations)


big loop: 0
big loop: 1
big loop: 2
big loop: 3
big loop: 4
big loop: 5
big loop: 6
big loop: 7
big loop: 8
big loop: 9
<class 'numpy.int64'>
<class 'numpy.int64'>


KeyError: 101031

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
from taskdataset import TaskDataset
import matplotlib.pyplot as plt
import os
import time
import pickle as pkl
from tqdm import tqdm
import argparse

from t2_functions import train, validate, l1_reg, lin_augment_affine

class RepresentationsDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        x, y = self.x[idx], self.y[idx]
        return torch.tensor(x), torch.tensor(y)

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
class Linear(nn.Module):
    def __init__(self, input_size, output_size):
        super(Linear, self).__init__()
        self.fc1 = nn.Linear(input_size, output_size)
    
    def forward(self, x):
        x = self.fc1(x)
        return x


criterion = nn.MSELoss()
batch_size = 32
epochs = 70
lr = 0.001

preds = []
ids = []

for i in range(10):
    print(f"big loop: {i}")
    folder = os.path.join('./data', 'test', 'affine', str(3), f'partition_{i}')
    with open(f'{folder}/A_train', 'rb') as f:
        A_train_reps = pkl.load(f)
    with open(f'{folder}/B_train', 'rb') as f:
        B_train_reps = pkl.load(f)
    with open(f'{folder}/B_test', 'rb') as f:
        B_test_reps = pkl.load(f)
    with open(f'{folder}/ids_train', 'rb') as f:
        ids_train = pkl.load(f)
    with open(f'{folder}/ids_test', 'rb') as f:
        ids_test = pkl.load(f)

    A_train_aug = torch.tensor(A_train_reps, dtype=torch.float32)
    B_train_aug = torch.tensor(B_train_reps, dtype=torch.float32)
    # A_train_aug, B_train_aug = lin_augment_affine(A_train_aug, B_train_aug, new_for_each_pair=1)

    train_dataset = RepresentationsDataset(x=A_train_aug, y=B_train_aug)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    lin_net = Linear(384, 384)
    lin_empty_net = Linear(384, 384)

    optim = torch.optim.Adam(lr=lr, params=lin_net.parameters())

    lin_last_net, lin_best_net = train(epochs, optim, criterion, l1_reg, train_loader, train_loader, lin_net, lin_empty_net, reg_lambda=0.0001)

    B_reps = np.concatenate([B_train_reps, B_test_reps], axis=0)
    B_reps = torch.tensor(B_reps, dtype=torch.float32)

    A_preds = lin_last_net(B_reps)
    A_preds = A_preds.detach().numpy()
    A_preds[:200] = np.array(A_train_reps)
    
    preds.append(A_preds)

    # to delete
    # ids_train = np.arange(B_train_reps.shape[0])
    # ids_test = np.arange(B_test_reps.shape[0])

    ids_iter = np.concatenate([ids_train, ids_test], axis=0)
    ids.append(ids_iter)

    # Convert the concatenated tensor to a NumPy array

preds = np.concatenate(preds, axis=0)
ids = np.concatenate(ids, axis=0)

# id_rep_map = {id: rep for id, rep in zip(ids, preds)}

dataset = torch.load("./data/SybilAttack.pt")
# representations = [id_rep_map[id] for id in dataset.ids]

# np.savez("./task2_submission.npz", ids=dataset.ids, representations=representations)
    


big loop: 0


  0%|          | 0/7 [00:00<?, ?it/s]/tmp/ipykernel_99252/3203303094.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(x), torch.tensor(y)
train | loss: 4.0051: 100%|██████████| 7/7 [00:00<00:00, 221.08it/s]


Epoch [1/70], Train Loss: 2.3871
Epoch [1/70], Val Loss: 1.0651


train | loss: 1.4424: 100%|██████████| 7/7 [00:00<00:00, 183.59it/s]

Epoch [2/70], Train Loss: 1.3906
Epoch [2/70], Val Loss: 0.9275


...

Epoch [68/70], Train Loss: 0.5593
Epoch [68/70], Val Loss: 0.4602


train | loss: 0.4751: 100%|██████████| 7/7 [00:00<00:00, 106.10it/s]


Epoch [69/70], Train Loss: 0.5377
Epoch [69/70], Val Loss: 0.4592


train | loss: 0.5299: 100%|██████████| 7/7 [00:00<00:00, 152.92it/s]

Epoch [70/70], Train Loss: 0.5386
Epoch [70/70], Val Loss: 0.4599


In [16]:
representations = []
for i, id in enumerate(dataset.ids):
    if id in id_rep_map:
        representations.append(id_rep_map[id])
    else:
        representations.append(preds[i])

np.savez("./task2_submission60.npz", ids=dataset.ids, representations=representations)

In [17]:
set1 = set(id_rep_map.keys())
set2 = set(dataset.ids)

print(len(set2 - set1))


17991
